# CS229 Assignment 3 - First Coding Question

We will be implementing the $l_1$-regularised least squares algorithm from question 3 of problem set 3 (https://see.stanford.edu/materials/aimlcs229/problemset3.pdf ) as part of Stanford's Engineering Everywhere CS229 Machine Learning course, taught by Andrew Ng in '08 ( https://see.stanford.edu/course/cs229 ).

In this question we will be minimising the cost function $$J(\theta)=\frac{1}{2}||X\theta - \vec{y}||_2^2 + \lambda ||\theta||_1,$$ where $X$ is our matrix of training examples, $\vec{y}$ is the vector of the correct classifications and $\theta$ is the parameter vector for our algorithm to learn. The $\lambda$ parameter contorls the amount of penilisation we want to apply. 

(For those who haven't studied it yet, the $||\cdot||_n$ is called a norm and is a way to measure the size of a vector in an abstract sense. Really what we care about here is the sum of the squared values of a vector, or the sum of the absolute values of a vector. Checkout for more details: https://machinelearningmastery.com/vector-norms-machine-learning/)

In [323]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We start by loading in our data. Note that $X$ is a $20\times 100$ sized matrix, and $Y$ is a $20\times 1$ vector, so that means we have a $100$ input features, with $20$ training pairs. Hence we have $\theta\in\mathbb{R}^{100}$.

In [324]:
xData=np.asmatrix(np.loadtxt("data/q3/x.dat"))
yData = np.transpose(np.asmatrix(np.loadtxt("data/q3/y.dat")))
m=xData.shape[0] # number of training examples
n=xData.shape[1] # number of features
converganceConst=0.00001

We also want to implement our cost function $J$. This is what we are aiming to minimise.

In [325]:
def objective(X, y, theta, lambdaConst):
    return 0.5*np.linalg.norm(X*theta-y)**2+lambdaConst*np.sum(np.absolute(theta))

To minimise we perform coordinate descent on $J$. To perform the coordinate descent we need to be able to calculate its derivative and set it to $0$. Unfortunately this is difficult when we have the $||\theta||_1=\sum|\theta_i|$ term there as this is not-differentiable in any of the $\theta_i$. 

The assignment has us rewrite $J$ cleverly to allow us to perform this differentiation, by extracting out the $\theta_i$ term and replacing the modulus with multiplication by sign($\theta_i$). Once we have done this, we perform the differentiation, and we see that we get $\theta_i$ updates to the following quantity



$$\theta_i = \frac{-X_i^T(X\bar{\theta}-\vec{y})-\lambda s_i}{X_i^TX_i},$$
where $X_i$ is the $i$th column of $X$, $\bar{\theta}$ is $\theta$ with the $i$th entry set to $0$ and $s_i$ is the sign of $\theta_i$. We first write a function to update a single $\theta_i$. This comes with the caveat that we have to take the max/min with $0$ and see which version then gives the smallest objective function to account for the introduction of the $s_i$ sign we did. (See the assignment solutions for more details on the maths here: https://see.stanford.edu/materials/aimlcs229/ps3_solution.pdf )

In [326]:
def updateSingleTheta(i, X, y, theta, lambdaConst):
    Xi=X[:,i]
    theta1, theta2, thetaBar=np.copy(theta), np.copy(theta), np.copy(theta)
    thetaBar[i]=0
    numerator1=-np.transpose(Xi)*(X*thetaBar-y)-lambdaConst
    numerator2=-np.transpose(Xi)*(X*thetaBar-y)+lambdaConst
    denominator=np.transpose(Xi)*Xi
    theta_i1=np.maximum(numerator1/denominator, 0)
    theta_i2=np.minimum(numerator2/denominator, 0)
    theta1[i]=theta_i1
    theta2[i]=theta_i2
    objective1=objective(X,y,theta1,lambdaConst)
    objective2=objective(X,y,theta2,lambdaConst)
    if (objective1 < objective2):
        return theta_i1
    else:
        return theta_i2

We now combine these individual update steps together to update the whole $\theta$ vector.

In [327]:
def updateTheta(X, y, theta, lambdaConst):
    diffs=[]
    for i in range(n):
        oldThetai=np.copy(theta[i])
        theta[i]=updateSingleTheta(i, X, y, theta, lambdaConst)
        diffs.append(abs(oldThetai-theta[i]))
    maxDiff=np.amax(diffs)
    return theta, maxDiff

We need to now write a function to repeat this updating until our $\theta$ converges. From the assgnment we are told to stop once all of the coordinates in one pass change by less than $10^{-5}$. We do this as part of the l1ls(X, y, lambda) function we are asked to create.

In [328]:
def l1ls(X, y, lambdaConst):
    iterate=True
    theta=np.ones((n,1))
    while(iterate):
        theta, maxDiff=updateTheta(X, y, theta, lambdaConst)
        if maxDiff < converganceConst:
            iterate=False
    return theta

Now we implement our hypothesis function, given a column vector $\theta$ of parameters and a column vector $x$ of input, we output $h_\theta(X)=\theta^T X = \theta_1x_1+\cdots+\theta_nx_n$ as our prediction. This is not used in the solution directly but it allows you to play with the predictions so I wanted to include it.

In [332]:
def predict(theta, X):
    return np.asscalar(np.transpose(theta)*X)

Finally we do as the assignment says and can run our algorithm for different $\lambda$ parameters. In particular the goal of this quesiton is to notice that when we set $\lambda=1$ we see that our predicted $\theta$ matches the sparsity pattern of the true $\theta$. i.e. it tells us exactly which features in the true $\theta$ were non-zero. In particular we could now use this as a feature identification algorithm. (You'll have to scroll a little here, sorry!)

In [330]:
theta = l1ls(xData, yData, 1)
print(theta)

[[ 0.49875625]
 [ 0.65610659]
 [-0.79057697]
 [-0.65564318]
 [-0.89191725]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [331]:
trueTheta = np.transpose(np.asmatrix(np.loadtxt("data/q3/theta.dat")))
print(trueTheta)

[[ 0.68372018]
 [ 0.84110202]
 [-0.83028605]
 [-0.85031124]
 [-0.93904984]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     